In [1]:
from compute import Var, Par, Model, adda, addf, ureg, Evaluable, buildidpvars, get_unit
from scipy import interpolate
import numpy as np
import openmdao.api as om
import sympy as sp

In [2]:
db2dec = lambda x: 10**(x/10)
dec2db = lambda x: 10*sp.log(sp.Abs(x), 10)

In [11]:
m = Model()

# Geometry
D_d = Var('D_d', 5, 'm')
D_f = Var('D_f', 5, 'm')
D_s = Var('D_s', 1, 'm')
t_d = Var('t_d', 0.5, 'm')
t_f = Var('t_f', 2, 'm')
t_s = Var('t_s', 6, 'm')

P_hotel = Par('P_hotel', 50, 'W')
eta_s = Par('eta_s', 0.27)
phi_s = Par('phi_s', 800, 'W/m**2')
theta_bar = Par('theta_bar', 55, 'deg')
I_d = Par('I_d', 0.9)
D = Par('D', 0.005)
L_s = Par('L_s', 10)
t_r = Var('t_r', unit='hr')
E_AUV = Par('E_AUV', 1.9, 'kW*hr')
t_service = Par('t_service', 12, 'hr')
t_comms = Par('t_comms', 1, 'hr')
gamma = Par('gamma', 2)
A_s = adda(m, 'A_s', 1.5*(D_f/2)**2, 'm**2')
t_mission = Par('t_mission', 24, 'hr')

# AUV recharging system
P_recharge = adda(m, 'P_recharge', 
    eta_s * phi_s * sp.cos(theta_bar) * I_d * (1-D)**L_s * A_s, 'W')
P_drawrecharge = adda(m, 'P_drawrecharge', P_hotel - P_recharge, 'W')
E_recharge_gen = adda(m, 'E_recharge_gen', P_recharge * t_r, 'W*hr')
E_service = adda(m, 'E_service', E_AUV * gamma, 'W*hr')
P_service = adda(m, 'P_service', E_service / t_service, 'W')
P_drawservice = adda(m, 'P_drawservice', P_hotel - P_service, 'W')

# Propulsion
rho = Par('rho', 1023.6, 'kg/m**3') # density of seawater [kg/m^3]
C_d = Par('C_d', 1) # estimate drag coefficient (a square flat plate at 90 deg to the flow is 1.17)
S_w = Par('S_w', 0.5, 'm**2') # will technically need to determine from A_solar and other structural needs... assume something for now
v = Var('v', 0.3, 'm/s') # [m/s]
eta_m = Par('eta_m', 0.75) # estimated, need to determine from motors?
P_move = adda(m, 'P_move', rho*C_d*S_w*v**3/(2*eta_m), 'W')

# Comms beginning
P_comms = Var('P_c', unit='W')

# Balances
E_required = adda(m, 'E_required', E_service + 
    (P_hotel + P_move)*t_mission + P_comms*t_comms, 'W*hr')

addf(m, E_recharge_gen-E_required)

# Battery
mu_batt = Par('mu_batt', 30, 'W*hr/kg')
DOD = Par('DOD', 0.4)
eta_trans = Par('eta_trans', 0.85)
nu_batt = Par('nu_batt', 450, 'kW*hr/(m**3)')
N = Par('N', 1)
C = adda(m, 'C', E_service/(DOD*N*eta_trans), 'kW*hr')
m_batt_zero = Par('m_batt_zero', 5, 'kg')
m_batt = adda(m, 'm_batt', C/mu_batt + m_batt_zero, 'kg')
V_batt = adda(m, 'V_batt', C/nu_batt, 'm**3')

# Comms
k = Par('k', 1.38065e-23, 'J/K')
c = Par('c', 3e8, 'm/s')
f = Par('f', 2.2, 'GHz')
lambd = adda(m, 'lambd', c/f, 'm')
eta_parab = Par('eta_parab', 0.55)
theta_t = Par('theta_t', 32)
error_t = Par('error_t', 27)
G_pt_db = adda(m, 'G_pt', 44.3-dec2db(theta_t**2))
L_pt_db = adda(m, 'L_pt_db', -12*(error_t/theta_t)**2)
G_t = adda(m, 'G_t', db2dec(G_pt_db+L_pt_db))
D_r = Par('D_r', 5.3, 'm')
G_rp = adda(m, 'G_rp', eta_parab*(np.pi*D_r/lambd)**2)
L_pr = Par('L_pr', db2dec(-0.1))
G_r = adda(m, 'G_r', G_rp*L_pr)
BER = Par('BER', 0.0001)
R = Par('R', 10.625, 'MB/s')
h = Par('h', 500, 'km')
Re = Par('Re', 6378, 'km')
T_s = Par('T_s', 135, 'K')
S = adda(m, 'S', sp.sqrt(h*(h+2*Re)), 'km')
L_a = Par('L_a', db2dec(-0.3))
L_sp = adda(m, 'L_sp', (lambd/(4*np.pi*S))**2)
L_l = Par('L_l', db2dec(-1))
EN = adda(m, 'EN', P_comms*L_a*L_sp*L_l*L_pr*G_r*G_t/(k*T_s*R))
EN_req = Par('EN_req', 30)

addf(m, EN-EN_req)

In [12]:
eqs, eqv, dout, dins = m.data_structures()

In [13]:
{elt for elt in dins if not elt.always_input}

{D_f, P_c, t_r, v}

In [ ]:
solver(groups, eqs, 0, 1, [t_r], [9])
explicit(groups, eqs, 1, A0, 1)
explicit(groups, eqs, 1, V0, 2)
explicit(groups, eqs, 1, rho_LG0, 3)
explicit(groups, eqs, 1, m_lg, 4)
explicit(groups, eqs, 1, m_t, 0)
explicit(groups, eqs, 1, L0, 5)
explicit(groups, eqs, 1, W0, 6)
explicit(groups, eqs, 1, D0, 7)

In [5]:
eqs

{0: (A_s, 0.375*D_f**2),
 1: (P_recharge, I_d*eta_s*phi_s*A_s*(1 - D)**L_s*cos(theta_bar)),
 2: (P_drawrecharge, P_hotel - P_recharge),
 3: (E_recharge_gen, P_recharge*t_r),
 4: (E_service, E_AUV*gamma),
 5: (P_service, E_service/t_service),
 6: (P_drawservice, P_hotel - P_service),
 7: (P_move, C_d*rho*S_w*V**3/(2*eta_m)),
 8: (E_required, t_comms*P_c + t_mission*(P_hotel + P_move) + E_service),
 9: (None, E_recharge_gen - E_required),
 10: (C, E_service/(DOD*N*eta_trans)),
 11: (m_batt, m_batt_zero + C/mu_batt),
 12: (V_batt, C/nu_batt),
 13: (lambd, c/f),
 14: (G_pt, -10*log(Abs(theta_t**2))/log(10) + 44.3),
 15: (L_pt_db, -12*error_t**2/theta_t**2),
 16: (G_t, 10**(G_pt/10 + L_pt_db/10)),
 17: (G_rp, 9.86960440108936*D_r**2*eta_parab/lambd**2),
 18: (G_r, L_pr*G_rp),
 19: (S, sqrt(h*(2*Re + h))),
 20: (L_sp, 0.00633257397764611*lambd**2/S**2),
 21: (EN, L_a*L_l*L_pr*G_r*G_t*L_sp*P_c/(R*T_s*k)),
 22: (None, -EN_req + EN)}

In [4]:

# # Hydrodynamics
# rho_s = var('rho_s', 650, 'kg/m**3')

# h_f = var('h_f', 
#     t_f*rho_s/rho+((D_s/D_f)**2*t_s+(D_d/D_f)**2*t_d)*(rho_s/rho-1), 'm')
# z_g = var('z_g', 
#     (t_f*D_f**2/2-t_s*D_s**2/2-(t_s+t_d/2)*D_d**2)/(D_f**2+D_f**2+D_s**2)-h_f, 
#     'm')
# #Vol = var('Vol', np.pi/4*(h_f*D_f**2+t_s*D_s**2+t_d*D_d**2), 'm**3')
# #GM = var('GM', D_f**4/(16*(D_f**2*h_f+D_s**2*t_s+D_d**2*t_d))-(h_f+t_s+t_d)/2, 'm')


# def debugfx():
#     print('stop')
